# Database Project

I will be analysing World Education Data.(https://ourworldindata.org/literacy)
<br> Data is in the form of csv(http://training.digitalvidya.com/wp-content/uploads/2018/09/Dataset-Assignment-2.zip)
<br> The data needs to be stored in the MySQL database using the procedure below.
<br> After successfully storing the data, I will be analysing this data.

Let's begin !

### Database Creation

In [1]:
import pymysql.cursors

In [2]:
class connection:
    def __init__(self, host, port, user, db):
        self.host = host
        self.port = port 
        self.user = user
        self.db= db       
        
    def connect(self):
        return pymysql.connect(host= self.host,
                                 port = self.port, 
                                 user = self.user,
                                 db= self.db)

In [3]:
c = connection('localhost', 3306,  'root', 'literacy database')

In [4]:
c

In [5]:
def execute(c, sqlQuery):
    conn = c.connect()
    try :
        with conn.cursor() as cursor:

            cursor.execute(sqlQuery)

        conn.commit()
    
    finally:
        print("connection closed")
        conn.close()

### Create Table

In [24]:
def execute_with_parameters(c, sql, subtitute_values, numberOfRows = 5):
    conn = c.connect()
    
    try:
        with conn.cursor() as cursor:
            
            cursor.execute(sql, subtitute_values)
            rows = cursor.fetchall()

            counter = 0
            for row in rows:
                
                counter += 1
                
                print(row)
                
                if counter == numberOfRows:
                    break
                    
                # conn is not autocommit by default. So you must commit to save
                # your changes.
        conn.commit()

    finally:
        print("conn closed")
        conn.close()

In [25]:
def convert_column_list(columnlist):
    s1 = ','.join(['`'+str(i)+'`' for i in columnlist]) 
    s2 = ','.join(['%s' for _ in columnlist]) 
    
    return s1,s2

In [26]:
def showTables(c):
    conn = c.connect()
    try:   
        with conn.cursor() as cursor:

            # SQL query string

            sqlQuery   = "show tables"   

            # Execute the sqlQuery

            cursor.execute(sqlQuery)

            rows = cursor.fetchall()

            for row in rows:

                print(row)

                # conn is not autocommit by default. So you must commit to save
                # your changes.

    finally:
        print("connection closed")
        conn.close()

In [27]:
showTables(c)

('dataset- assignment - 1',)
('literacy_dataset',)
connection closed


In [28]:
def insert(c, tablename, columns, values,subtitute_values):
    conn = c.connect()
    try:
        with conn.cursor() as cursor:
            
            # SQL query string
            sql = "INSERT INTO `"+ tablename + "` (" + columns + " ) " + " VALUES " + "(" + values + ");"
            print(sql) 
            print(subtitute_values)
            cursor.execute(sql, subtitute_values)
        conn.commit()

    finally:
        print("conn closed")
        conn.close()

Try inserting a row to test the insert() function.

In [29]:
columns = ['entity','code','year','Elderly literacy rate','Youth literacy rate', 'Total population']
x = ('Afghanistan','AFG',1800,0,0,3280000)

In [30]:
columns,values = convert_column_list(columns)
insert(c,"literacy_dataset",columns,values,x)

INSERT INTO `literacy_dataset` (`entity`,`code`,`year`,`Elderly literacy rate`,`Youth literacy rate`,`Total population` )  VALUES (%s,%s,%s,%s,%s,%s);
('Afghanistan', 'AFG', 1800, 0, 0, 3280000)
conn closed


In [31]:
def showContents(c,sql, numberOfRows = 5):
    conn = c.connect()
    try:   
        with conn.cursor() as cursor: 

            # Execute the sqlQuery
            
            cursor.execute(sql)

            rows = cursor.fetchall()

            counter = 0
            for row in rows:
                
                counter += 1
                
                print(row)
                
                if counter == numberOfRows:
                    break
                    
                # conn is not autocommit by default. So you must commit to save
                # your changes.

    finally:
        print("conn closed")
        conn.close()

In [32]:
sql = "Select * from literacy_dataset"

showContents(c, sql)

('Afghanistan', 'AFG', 1800, Decimal('0.00000'), Decimal('0.00000'), 3280000)
('Afghanistan', 'AFG', 1820, Decimal('0.00000'), Decimal('0.00000'), 3280000)
('Afghanistan', 'AFG', 1870, Decimal('0.00000'), Decimal('0.00000'), 4207000)
('Afghanistan', 'AFG', 1913, Decimal('0.00000'), Decimal('0.00000'), 5730000)
('Afghanistan', 'AFG', 1950, Decimal('0.00000'), Decimal('0.00000'), 8151455)
conn closed


### Load data from csv to database

In [33]:
import csv

def csvToDB(c,csv_name,columns,tablename):
    with open(csv_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        columns,values = convert_column_list(columns)

        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(row)
                line_count += 1
            else:
                conn = c.connect()
                insert(conn,tablename,columns,values,row)
                line_count += 1
        print(f'Inserted {line_count} lines.')

In [34]:
## START CODE HERE
csvToDB(c, r"C:\Users\sanjay\Desktop\Python\Dataset- Assignment - 1.csv",columns,'literacy_dataset')
## END CODE HERE

['Entity', 'Code', 'Year', 'Elderly literacy rate, population 65+ years, both sexes (%) (%)', 'Youth literacy rate, population 15-24 years, both sexes (%) (%)', 'Total population (Gapminder)']
conn closed


AttributeError: 'NoneType' object has no attribute 'close'

In the above cell you will get warning, ignore it after calling csvToDB().
<br> The warning appears because the value for 'Elderly literacy rate, population 65+ years, both sexes' and 'Youth literacy rate, population 15-24 years, both sexes' is missing in many rows.

In [35]:
sql = '''Select * from literacy_dataset'''
showContents(c, sql)

('Afghanistan', 'AFG', 1800, Decimal('0.00000'), Decimal('0.00000'), 3280000)
('Afghanistan', 'AFG', 1820, Decimal('0.00000'), Decimal('0.00000'), 3280000)
('Afghanistan', 'AFG', 1870, Decimal('0.00000'), Decimal('0.00000'), 4207000)
('Afghanistan', 'AFG', 1913, Decimal('0.00000'), Decimal('0.00000'), 5730000)
('Afghanistan', 'AFG', 1950, Decimal('0.00000'), Decimal('0.00000'), 8151455)
conn closed


Wherever you can see Decimal('0E-10') those are null values.
<br>Now that the csv is converted to Database you can test your knowledge of using SQL commands to fetch data.

## What is the youth literacy rate of different countries during 1913?

In [66]:
## START CODE HERE
sql = '''select `entity`,`Youth Literacy Rate` from literacy_dataset where `year` = 1913;'''
showContents(c, sql)
## END CODE HERE

('Afghanistan', Decimal('0.00000'))
('Afghanistan', Decimal('0.00000'))
('Albania', Decimal('0.00000'))
('Algeria', Decimal('0.00000'))
('Argentina', Decimal('0.00000'))
conn closed


<table>
  <tr>
  <td>**Expected Output:**<td>
    <td style="text-align:left">
('Afghanistan', Decimal('0E-10'))
<br>('Albania', Decimal('0E-10'))
<br>('Algeria', Decimal('0E-10'))
<br>('Argentina', Decimal('0E-10'))
<br>('Australia', Decimal('0E-10'))
<br>conn closed</td>
  </tr>
</table>


### Which country has the highest elderly literacy rate during 2015?

In [64]:
## START CODE HERE
sql = '''SELECT `entity`,`elderly literacy rate` FROM literacy_dataset 
where `Year` = 2015 
order by `elderly literacy rate` DESC LIMIT 1;'''
showContents(c,sql)
## END CODE HERE

('North Korea', Decimal('99.99609'))
conn closed


<table>
  <tr>
  <td>**Expected Output:**<td>
    <td style="text-align:left">
('North Korea', Decimal('99.9960937500'))
<br> conn closed
</td>
  </tr>
</table>

### Which country has highest population?

In [65]:
## START CODE HERE
sql = '''select `entity`,`Total Population` from literacy_dataset order by `Total Population` DESC LIMIT 1;'''
showContents(c,sql)
## END CODE HERE

('China', 1359368470)
conn closed


<table>
  <tr>
  <td>**Expected Output:**<td>
    <td style="text-align:left">
('China', 1359368470)
<br>conn closed
</td>
  </tr>
</table>

### For each year which were the highest populated countries and what was their elderly and youth literacy rates?

In [58]:
## START CODE HERE
sql ='''select `year`,`Total Population`,`Elderly Literacy Rate`,
`Youth Literacy Rate` from literacy_dataset 
where `year` in (2013,2012,2011,2010,2009)
group by `year`;'''
showContents(c,sql)
## END CODE HERE

(2009, 30577756, Decimal('0.00000'), Decimal('0.00000'))
(2010, 31411743, Decimal('0.00000'), Decimal('0.00000'))
(2011, 32358260, Decimal('20.32185'), Decimal('46.99005'))
(2012, 33397058, Decimal('0.00000'), Decimal('0.00000'))
(2013, 34499915, Decimal('0.00000'), Decimal('0.00000'))
conn closed


<table>
  <tr>
  <td>**Expected Output:**<td>
    <td style="text-align:left">
(2013, Decimal('0E-10'), Decimal('0E-10'), 34499915)
<br>(2012, Decimal('0E-10'), Decimal('0E-10'), 33397058)
<br>(2011, Decimal('20.3218498230'), Decimal('46.9900512695'), 32358260)
<br>(2010, Decimal('0E-10'), Decimal('0E-10'), 31411743)
<br>(2009, Decimal('0E-10'), Decimal('0E-10'), 30577756)
<br> conn closed
</td>
  </tr>
</table>

### Show records of BRICS (Brazil, Russia, India, China,South Africa) countries only 

In [40]:
## START CODE HERE
sql = "select * from literacy_dataset where `entity` = 'Brazil' or `entity` = 'Russia' or `entity` = 'India' or `entity` = 'China' or `entity` = 'South Africa';"
showContents(c,sql)
## END CODE HERE

('Brazil', 'BRA', 1700, Decimal('0.00000'), Decimal('0.00000'), 1250000)
('Brazil', 'BRA', 1800, Decimal('0.00000'), Decimal('0.00000'), 3639636)
('Brazil', 'BRA', 1820, Decimal('0.00000'), Decimal('0.00000'), 4507000)
('Brazil', 'BRA', 1850, Decimal('0.00000'), Decimal('0.00000'), 7234000)
('Brazil', 'BRA', 1870, Decimal('0.00000'), Decimal('0.00000'), 9797000)
conn closed


<table>
  <tr>
  <td>**Expected Output:**<td>
    <td style="text-align:left">
('Brazil', 'BRA', 1700, Decimal('0E-10'), Decimal('0E-10'), 1250000)
<br>('Brazil', 'BRA', 1800, Decimal('0E-10'), Decimal('0E-10'), 3639636)
<br>('Brazil', 'BRA', 1820, Decimal('0E-10'), Decimal('0E-10'), 4507000)
<br>('Brazil', 'BRA', 1850, Decimal('0E-10'), Decimal('0E-10'), 7234000)
<br>('Brazil', 'BRA', 1870, Decimal('0E-10'), Decimal('0E-10'), 9797000)
<br>('Brazil', 'BRA', 1871, Decimal('0E-10'), Decimal('0E-10'), 9980000)
<br>('Brazil', 'BRA', 1872, Decimal('0E-10'), Decimal('0E-10'), 10167000)
<br>('Brazil', 'BRA', 1873, Decimal('0E-10'), Decimal('0E-10'), 10358000)
<br>('Brazil', 'BRA', 1874, Decimal('0E-10'), Decimal('0E-10'), 10552000)
<br>('Brazil', 'BRA', 1875, Decimal('0E-10'), Decimal('0E-10'), 10749000)
<br> conn closed
</td>
  </tr>
</table>

### What is the highest elderly literacy rate of each of the BRICS countries?

In [54]:
## START CODE HERE
sql = '''select `entity`,max(`Elderly Literacy Rate`) from literacy_dataset where `entity` = 'Brazil' or `entity` = 'Russia' or `entity` = 'India' or `entity` = 'China' or `entity` = 'South Africa' group by `entity`;'''
showContents(c,sql)
## END CODE HERE

('Brazil', Decimal('74.19575'))
('China', Decimal('81.28252'))
('India', Decimal('42.70036'))
('Russia', Decimal('99.47242'))
('South Africa', Decimal('71.25617'))
conn closed


<table>
  <tr>
  <td>**Expected Output:**<td>
    <td style="text-align:left">
('Brazil', Decimal('74.1957473755'))
<br>('China', Decimal('81.2825164795'))
<br>('India', Decimal('42.7003593445'))
<br>('Russia', Decimal('99.4724197388'))
<br>('South Africa', Decimal('71.2561721802'))
<br> conn closed
</td>
  </tr>
</table>

### What is the highest youth literacy rate of each of the BRICS countries?

In [56]:
## START CODE HERE
sql = '''select `entity`,max(`Youth Literacy Rate`) from literacy_dataset where `entity` in ('Brazil','Russia','India','China','South Africa')group by `entity`;'''
showContents(c,sql)
## END CODE HERE

('Brazil', Decimal('98.93239'))
('China', Decimal('99.72886'))
('India', Decimal('89.66406'))
('Russia', Decimal('99.72460'))
('South Africa', Decimal('99.02700'))
conn closed


<table>
  <tr>
  <td>**Expected Output:**<td>
    <td style="text-align:left">
('Brazil', Decimal('98.9323883057'))
<br>('China', Decimal('99.7288589478'))
<br>('India', Decimal('89.6640625000'))
<br>('Russia', Decimal('99.7246017456'))
<br>('South Africa', Decimal('99.0270004272'))
<br> conn closed
</td>
  </tr>
</table>